# Profile PreProcessing

## 1. Gray Filter
## 2. Cropping
## 3. Binary Inversion
## 4. Integration or Individuall OCR

스크린샷 위치 : C:\Users\KH_Home\Nox_share\ImageShare\Screenshots

tesseract_path : C:\Program Files\Tesseract-OCR\tesseract.exe

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import pandas as pd

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [ ]:
profile_path = './Screenshots/'
profile_list = glob.glob(profile_path + '*.png')
for fn in profile_list:
    print(fn)

In [ ]:
crop_dict = {
    'ID' : np.array([185, 216, 595, 672], dtype=int),
    'power' : np.array([290, 320, 690, 880], dtype=int),
    'kill' : np.array([290, 320, 880, 1050], dtype=int),
}

In [ ]:
def bin_inv(img, thr=140, show=False):
    ret,thresh2 = cv2.threshold(img,thr,255,cv2.THRESH_BINARY_INV)
    ocr_result = pytesseract.image_to_string(thresh2)
    if show:
        plt.imshow(thresh2, 'gray')
        plt.show()
        print(ocr_result)
    return thresh2, ocr_result

In [ ]:
def str2num(strnum):
    result = strnum.replace(',', '').replace('.', '')
    if result.isdigit():
        return int(result)
    else:
        return 0

In [ ]:
class Player():
    def __init__(self, ID, power, kill):
        self.ID = ID
        self.power = power
        self.kill = kill
        self.name = None
        self.path = None

# Load id2name.xlsx to dict

In [ ]:
def load_id2name(file_path):
    df_id2name = pd.read_excel(file_path)
    id2name = dict([])

    for i in range(len(df_id2name)):
        row = df_id2name.loc[i]
        id2name[row['ID']] = row['name']
    
    return id2name

In [ ]:
id2name = load_id2name('id2name.xlsx')
id2name

# Let's OCR

In [ ]:
Players = dict([])

L = len(profile_list)
for i, fn in enumerate(profile_list):
    print('{}/{}'.format(i,L))
    img = cv2.imread(fn,0)

    IDp = crop_dict['ID']
    powerp = crop_dict['power']
    killp = crop_dict['kill']
    
    img_ID = img[IDp[0]:IDp[1], IDp[2]:IDp[3]]
    img_power = img[powerp[0]:powerp[1], powerp[2]:powerp[3]]
    img_kill = img[killp[0]:killp[1], killp[2]:killp[3]]
    
    _, str_ID = bin_inv(img_ID)
    _, str_power = bin_inv(img_power)
    _, str_kill = bin_inv(img_kill)
    
    ID, power, kill = str2num(str_ID), str2num(str_power), str2num(str_kill)
    
    Players[ID] = Player(ID,power,kill)        
    Players[ID].path = fn.split('\\')[-1]
    
    try:
        Players[ID].name = id2name[ID]
    except KeyError:
        print('KeyError!! : checkt {}'.format(Players[ID].path))

In [ ]:
df = {
    'ID' : [],
    'name' : [],
    'power' : [],
    'kill' : [],
    'path' : []
}
df = pd.DataFrame(df, dtype=int)

L = len(Players)
for ID, p in Players.items():
    df = df.append({
        'ID' : ID,
        'name' : p.name,
        'power' : p.power,
        'kill' : p.kill,
        'path' : p.path
    }, ignore_index=True)

In [ ]:
df

# save as xlsx

In [ ]:
df.to_excel('tmp.xlsx', index=False)

# Matching ID -name

# 여긴 이제 안씀

# id name matching

In [ ]:
df_id2name = pd.read_excel('id2name_00.xlsx')
id2name = dict([])
for i in range(len(df_id2name)):
    df_row = df_id2name.loc[i]
    id2name[df_row['ID']] = df_row['name']

In [ ]:
id2name

# dict to xlsx

In [ ]:
df_xlsx = {
    'ID' : [],
    'name' : [],
}
df_xlsx = pd.DataFrame(df_xlsx, dtype=int)

for ID, name in id2name.items():
    df_xlsx = df_xlsx.append({
        'ID' : ID,
        'name' : name
    }, ignore_index=True)
    
df_xlsx.to_excel('id2name.xlsx', index=False)

# ALL

In [ ]:
# for fn in profile_list:
#     img = cv2.imread(fn,0)
#     plt.imshow(img, 'gray')
#     plt.show()

#     IDp = crop_dict['ID']
#     powerp = crop_dict['power']
#     killp = crop_dict['kill']
    
#     img_ID = img[IDp[0]:IDp[1], IDp[2]:IDp[3]]
#     img_power = img[powerp[0]:powerp[1], powerp[2]:powerp[3]]
#     img_kill = img[killp[0]:killp[1], killp[2]:killp[3]]
    
#     i1 = bin_inv(img_ID, show=True)
#     i2 = bin_inv(img_power, show=True)
#     i3 = bin_inv(img_kill, show=True)
    

#     a = input('y or n')
#     if a == 'n':
#         break